In [171]:
from numpy import unique
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel
from sklearn.feature_selection import f_regression, f_classif
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix)
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
#import scikitplot as skplt
import plotly.graph_objs as go
import plotly.offline as py
from sklearn import ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix)
from mlxtend.plotting import plot_confusion_matrix

In [172]:
# 讀取 training data 並過濾 NA 欄位
df = pd.read_csv("./house-prices-advanced-regression-techniques/train.csv")#,index_col=0)
print(df.shape)
df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [176]:
df = df.drop(df[(df['GrLivArea']>4000) & (df['SalePrice']<200000)].index)

In [177]:
df.shape

(1458, 81)

In [178]:
df_bind1 = df[['SalePrice']] ## 記錄答案 # answer
df_bind1.shape

(1458, 1)

In [179]:
## Training set 的 feature column names
feature_names = list(df.columns)
print("df.shape:",df.shape) # 處理過後有的行列數量 # 包含 cell_name 和 ans
print("df_bind1.shape",df_bind1.shape)

df.shape: (1458, 81)
df_bind1.shape (1458, 1)


In [180]:
#Creating dataframe with percentages of missing values
df_concat_na = (df.isnull().sum() / len(df)) * 100

#Sorting by percentages of missing values
df_concat_na = df_concat_na.sort_values(ascending=False)[:20] #values of missing percentages after index 20 drop to around 0.03.. 

missing_values = pd.DataFrame({'Missing %' :df_concat_na})
missing_values.head()

,Missing %
PoolQC,99.588477
MiscFeature,96.296296
Alley,93.758573
Fence,80.727023
FireplaceQu,47.325103


In [181]:
df['PoolQC'].fillna('None', inplace=True)
df['MiscFeature'].fillna('None', inplace=True)
df['Alley'].fillna('None', inplace=True)
df['Fence'].fillna('None', inplace=True)
df['FireplaceQu'].fillna('None', inplace=True)
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    df[col].fillna('None', inplace=True)
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    df[col].fillna(0, inplace=True)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):    
    df[col].fillna('None', inplace=True)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtHalfBath', 'BsmtFullBath'):
    df[col].fillna(0, inplace=True)
df['MasVnrType'].fillna('None', inplace=True)
df['MasVnrArea'].fillna(0, inplace=True)
df['MSZoning'].fillna('RL', inplace=True)
df['Utilities'].value_counts()
df.drop(['Utilities'], axis=1, inplace=True)
df['Functional'].fillna('Typ', inplace=True)
df['Electrical'].fillna('SBrkr', inplace=True)
df['KitchenQual'].fillna('TA', inplace=True)
df['Exterior1st'].fillna('TA', inplace=True)
df['Exterior2nd'].fillna('TA', inplace=True)
df['SaleType'].fillna('WD', inplace=True)

In [182]:
#Creating dataframe with percentages of missing values
df_concat_na = (df.isnull().sum() / len(df)) * 100

#Sorting by percentages of missing values
df_concat_na = df_concat_na.sort_values(ascending=False) 

missing_values = pd.DataFrame({'Missing %' :df_concat_na})
missing_values.head()

,Missing %
SalePrice,0.0
SaleCondition,0.0
RoofMatl,0.0
Exterior1st,0.0
Exterior2nd,0.0


In [183]:
df.shape

(1458, 80)

In [184]:
df = df.drop(['SalePrice','Id'],axis=1)
df_train = df
X = df_train
y = df_bind1['SalePrice']
print(X.shape)
print(y.shape)

(1458, 78)
(1458,)


In [185]:
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,0,None,None,None,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,0,None,None,None,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,0,None,None,None,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,0,None,None,None,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,0,None,None,None,0,12,2008,WD,Normal


In [186]:
#MSSubClass
df_train['MSSubClass'].astype(str)


#Changing OverallCond into a categorical variable
df_train['OverallCond'] = df_train['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
# df_train['YrSold'] = df_train['YrSold'].astype(str)
# df_train['MoSold'] = df_train['MoSold'].astype(str)

In [187]:
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,0,None,None,None,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,0,None,None,None,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,0,None,None,None,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,0,None,None,None,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,0,None,None,None,0,12,2008,WD,Normal


In [188]:
numeric_col_list = ['LotArea','YearBuilt','YearRemodAdd',
                    'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF' , '2ndFlrSF' , 
                    'LowQualFinSF' , 'GrLivArea' , 'BsmtFullBath' , 'BsmtHalfBath' , 'FullBath' , 
                    'HalfBath' , 'TotRmsAbvGrd', 'Fireplaces' , 
                    'GarageCars' , 'GarageArea' , 'WoodDeckSF' , 'OpenPorchSF' , 'EnclosedPorch' , 
                    '3SsnPorch' , 'ScreenPorch' , 'PoolArea' , 'MiscVal' ,
                    'MasVnrArea','GarageYrBlt','LotFrontage','MSSubClass', 'MoSold' , 'YrSold'] 
print(len(numeric_col_list))
df_one_hot =  df_train.drop(numeric_col_list, axis=1)
df_one_hot.head()

32


,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,...,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [189]:
df_train = df_train[numeric_col_list]
df_train.head()

,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MasVnrArea,GarageYrBlt,LotFrontage,MSSubClass,MoSold,YrSold
0,8450,2003,2003,706,0,150,856,856,854,0,...,0,0,0,0,196.0,2003.0,65.0,60,2,2008
1,9600,1976,1976,978,0,284,1262,1262,0,0,...,0,0,0,0,0.0,1976.0,80.0,20,5,2007
2,11250,2001,2002,486,0,434,920,920,866,0,...,0,0,0,0,162.0,2001.0,68.0,60,9,2008
3,9550,1915,1970,216,0,540,756,961,756,0,...,0,0,0,0,0.0,1998.0,60.0,70,2,2006
4,14260,2000,2000,655,0,490,1145,1145,1053,0,...,0,0,0,0,350.0,2000.0,84.0,60,12,2008


In [191]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in list(df_one_hot.columns):
    df_one_hot[i] = labelencoder.fit_transform(df_one_hot[i])

df_one_hot.head()

,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,1,1,3,3,4,0,5,2,2,...,1,2,5,5,2,3,4,1,8,4
1,3,1,1,3,3,2,0,24,1,2,...,1,2,5,5,2,3,4,1,8,4
2,3,1,1,0,3,4,0,5,2,2,...,1,2,5,5,2,3,4,1,8,4
3,3,1,1,0,3,0,0,6,2,2,...,5,3,5,5,2,3,4,1,8,0
4,3,1,1,0,3,2,0,15,2,2,...,1,2,5,5,2,3,4,1,8,4


In [192]:
df_train_ = pd.concat([df_train, df_one_hot], axis=1)
df_train_.head()

,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,2003,2003,706,0,150,856,856,854,0,...,1,2,5,5,2,3,4,1,8,4
1,9600,1976,1976,978,0,284,1262,1262,0,0,...,1,2,5,5,2,3,4,1,8,4
2,11250,2001,2002,486,0,434,920,920,866,0,...,1,2,5,5,2,3,4,1,8,4
3,9550,1915,1970,216,0,540,756,961,756,0,...,5,3,5,5,2,3,4,1,8,0
4,14260,2000,2000,655,0,490,1145,1145,1053,0,...,1,2,5,5,2,3,4,1,8,4


In [193]:
X = df_train_
y = df_bind1['SalePrice']
print(X.shape)
print(y.shape)

(1458, 78)
(1458,)


In [194]:
one_hot_col = list(df_train_.columns)
len(one_hot_col)

78

In [195]:
len(set(one_hot_col))

78

# Hyper Parameter 調整

In [200]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats

# evaluate a given model by making predictions on X_valid
def get_v_score(model):
    valid_predictions=model.predict(test_X)
    score=np.sqrt(mean_squared_error(test_y, valid_predictions))
    return score

# Models name
models_name=['RandomForestRegressor',
             'GradientBoostingRegressor',
             'XGBRegressor',
             'Lasso']
# Models
models=[RandomForestRegressor(),
        GradientBoostingRegressor(),
        XGBRegressor(),
        Lasso(alpha=0.0005,tol=0.001)]

In [201]:
# train_data = df_train_tree
train_data = df_train_

# 建立訓練與測試資料
train_X, test_X, train_y, test_y = train_test_split(train_data, y, test_size = 0.3)


#Fit and get scores for each model
scores_list=[]
for model in models:
    model.fit(train_X,train_y)
    scores_list.append(get_v_score(model))
    
#Convert list to dataframe
data={'Model':models_name,'RMSE': scores_list}
scores_df=pd.DataFrame(data)
#Sort by valid RMSLE
scores_df.sort_values(by='RMSE').round(5).style.set_properties(**{'background-color': 'black','color': 'white'})

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:11:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Model,RMSE
1,GradientBoostingRegressor,21976.3
2,XGBRegressor,22587.5
3,Lasso,24754.5
0,RandomForestRegressor,24763.5


In [202]:
df_train_.shape

(1458, 78)

In [203]:
# train_data = df_train_tree
train_data = df_train_

In [204]:
# 建立訓練與測試資料
train_X, test_X, train_y, test_y = train_test_split(train_data, y, test_size = 0.2)

In [273]:
score = []
error = []
time = []
# 建立 random forest 模型
for i in range(80,120,2):
    time.append(i)
    reg = XGBRegressor(random_state = 0,n_estimators =i)#,min_child_weight=i)
    reg.fit(train_X, train_y)
    # 預測
    test_y_predicted = reg.predict(test_X)
    # 績效
    scores = reg.score(test_X, test_y)
    score.append(scores)
    errors = mean_squared_error(test_y, test_y_predicted)
    error.append(errors)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:23:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:23:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



...

[23:23:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:23:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:23:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [278]:
score = []
error = []
time = []
# 建立 random forest 模型
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]:
    time.append(i)
    reg = XGBRegressor(random_state = 0,n_estimators =100,min_child_weight=i)
    reg.fit(train_X, train_y)
    # 預測
    test_y_predicted = reg.predict(test_X)
    # 績效
    scores = reg.score(test_X, test_y)
    score.append(scores)
    errors = mean_squared_error(test_y, test_y_predicted)
    error.append(errors)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:25:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:25:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



...

[23:25:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:25:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:25:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [280]:
score = []
error = []
time = []
# 建立 random forest 模型
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]:
    time.append(i)
    reg = XGBRegressor(random_state = 0,n_estimators =100,min_child_weight=3,max_depth = i)
    reg.fit(train_X, train_y)
    # 預測
    test_y_predicted = reg.predict(test_X)
    # 績效
    scores = reg.score(test_X, test_y)
    score.append(scores)
    errors = mean_squared_error(test_y, test_y_predicted)
    error.append(errors)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:26:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:26:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



...

[23:26:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:26:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[23:26:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [281]:
score

[0.8522450089212313,
 0.8971219537430986,
 0.9094930622784864,
 0.8997537404380659,
 0.8969060783674105,
 0.8949084924844428,
 0.8932888094365334,
 0.8871585634650097,
 0.8877455140733865,
 0.8877629206447497,
 0.8868193334882273,
 0.8890542478134186,
 0.8895226014305625,
 0.8876531912537542,
 0.8873568690741165,
 0.8869709283071072,
 0.88883307432389,
 0.8864142170401619,
 0.8861571799423686]

In [275]:
error

[722734987.2278922,
 720503229.2993739,
 719420150.3641499,
 716738837.5352905,
 717224611.7936729,
 715055030.1164672,
 714696795.413153,
 710409993.5828791,
 706976789.7641596,
 705995604.1422046,
 703752043.678881,
 705497503.4316446,
 707212753.5068626,
 706628194.7938255,
 707703862.621898,
 707784830.1929657,
 709146573.2457587,
 709004533.840716,
 708576687.9659574,
 708480964.9396553]

In [295]:
# 讀取 training data 並過濾 NA 欄位
df = pd.read_csv("./house-prices-advanced-regression-techniques/test.csv")#,index_col=0)
print(df.shape)
df.head()

(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [296]:
df_id = df['Id']
df_id.shape

(1459,)

In [297]:
#Creating dataframe with percentages of missing values
df_concat_na = (df.isnull().sum() / len(df)) * 100

#Sorting by percentages of missing values
df_concat_na = df_concat_na.sort_values(ascending=False) 

missing_values = pd.DataFrame({'Missing %' :df_concat_na})
missing_values.head(20)

,Missing %
PoolQC,99.794380
MiscFeature,96.504455
Alley,92.666210
Fence,80.123372
FireplaceQu,50.034270
LotFrontage,15.558602
GarageCond,5.346127
GarageQual,5.346127
GarageYrBlt,5.346127
GarageFinish,5.346127


In [298]:
df['PoolQC'].fillna('None', inplace=True)
df['MiscFeature'].fillna('None', inplace=True)
df['Alley'].fillna('None', inplace=True)
df['Fence'].fillna('None', inplace=True)
df['FireplaceQu'].fillna('None', inplace=True)
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    df[col].fillna('None', inplace=True)
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    df[col].fillna(0, inplace=True)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):    
    df[col].fillna('None', inplace=True)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtHalfBath', 'BsmtFullBath'):
    df[col].fillna(0, inplace=True)
df['MasVnrType'].fillna('None', inplace=True)
df['MasVnrArea'].fillna(0, inplace=True)
df['MSZoning'].fillna('RL', inplace=True)
df['Utilities'].value_counts()
df.drop(['Utilities'], axis=1, inplace=True)
df['Functional'].fillna('Typ', inplace=True)
df['Electrical'].fillna('SBrkr', inplace=True)
df['KitchenQual'].fillna('TA', inplace=True)
df['Exterior1st'].fillna('TA', inplace=True)
df['Exterior2nd'].fillna('TA', inplace=True)
df['SaleType'].fillna('WD', inplace=True)

In [299]:
df = df.drop(['Id'],axis=1)
X = df
print(X.shape)

#MSSubClass
df['MSSubClass'].astype(str)


#Changing OverallCond into a categorical variable
df['OverallCond'] = df['OverallCond'].astype(str)


(1459, 78)


In [300]:
numeric_col_list = ['LotArea','YearBuilt','YearRemodAdd',
                    'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF' , '2ndFlrSF' , 
                    'LowQualFinSF' , 'GrLivArea' , 'BsmtFullBath' , 'BsmtHalfBath' , 'FullBath' , 
                    'HalfBath' , 'TotRmsAbvGrd', 'Fireplaces' , 
                    'GarageCars' , 'GarageArea' , 'WoodDeckSF' , 'OpenPorchSF' , 'EnclosedPorch' , 
                    '3SsnPorch' , 'ScreenPorch' , 'PoolArea' , 'MiscVal' ,
                    'MasVnrArea','GarageYrBlt','LotFrontage','MSSubClass', 'MoSold' , 'YrSold'] 
df_one_hot =  df.drop(numeric_col_list, axis=1)
df_one_hot.head()

,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RH,Pave,None,Reg,Lvl,Inside,Gtl,NAmes,Feedr,Norm,...,Attchd,Unf,TA,TA,Y,None,MnPrv,None,WD,Normal
1,RL,Pave,None,IR1,Lvl,Corner,Gtl,NAmes,Norm,Norm,...,Attchd,Unf,TA,TA,Y,None,None,Gar2,WD,Normal
2,RL,Pave,None,IR1,Lvl,Inside,Gtl,Gilbert,Norm,Norm,...,Attchd,Fin,TA,TA,Y,None,MnPrv,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,Inside,Gtl,Gilbert,Norm,Norm,...,Attchd,Fin,TA,TA,Y,None,None,None,WD,Normal
4,RL,Pave,None,IR1,HLS,Inside,Gtl,StoneBr,Norm,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [301]:
df_one_hot.shape

(1459, 46)

In [302]:
df_train = df[numeric_col_list]
df_train.head()

,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MasVnrArea,GarageYrBlt,LotFrontage,MSSubClass,MoSold,YrSold
0,11622,1961,1961,468.0,144.0,270.0,882.0,896,0,0,...,0,120,0,0,0.0,1961.0,80.0,20,6,2010
1,14267,1958,1958,923.0,0.0,406.0,1329.0,1329,0,0,...,0,0,0,12500,108.0,1958.0,81.0,20,6,2010
2,13830,1997,1998,791.0,0.0,137.0,928.0,928,701,0,...,0,0,0,0,0.0,1997.0,74.0,60,3,2010
3,9978,1998,1998,602.0,0.0,324.0,926.0,926,678,0,...,0,0,0,0,20.0,1998.0,78.0,60,6,2010
4,5005,1992,1992,263.0,0.0,1017.0,1280.0,1280,0,0,...,0,144,0,0,0.0,1992.0,43.0,120,1,2010


In [303]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in list(df_one_hot.columns):
    df_one_hot[i] = labelencoder.fit_transform(df_one_hot[i])

df_one_hot.shape

(1459, 46)

In [304]:
df_train.shape

(1459, 32)

In [305]:
df_train_ = pd.concat([df_train, df_one_hot], axis=1)
df_train_.shape

(1459, 78)

In [306]:
df_test = df_train_

df_test.shape

(1459, 78)

In [307]:
X = df_test

In [308]:
reg = XGBRegressor(random_state = 0,n_estimators =100,min_child_weight=3,max_depth = 3)#random_state=1,n_estimators =100)
reg.fit(train_X, train_y)
test_y_predicted = reg.predict(X)

[23:28:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [309]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import make_scorer,r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, RobustScaler
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [310]:
reg = XGBRegressor(random_state = 0,n_estimators =100,min_child_weight=3,max_depth = 3)#random_state=0,n_estimators =100)
reg.fit(train_X, train_y)
test_y_predicted = reg.predict(X)

[23:29:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [311]:
test_y_predicted

array([122839.29, 166805.44, 180670.75, ..., 173781.83, 120759.99,
       238652.44], dtype=float32)

In [312]:
df_ans = pd.DataFrame({'Id': df_id, 'SalePrice': test_y_predicted})
df_ans.set_index("Id" , inplace=True)
df_ans.head()

,SalePrice
Id,
1461,122839.289062
1462,166805.437500
1463,180670.750000
1464,176208.406250
1465,202182.781250


In [313]:
df_ans.to_csv("ans.csv")

In [314]:
df_ans.shape

(1459, 1)